In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import mplfinance as mpf
from pyclustering.cluster.silhouette import silhouette_ksearch_type, silhouette_ksearch
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from perceptually_important import find_pips

In [6]:
data = pd.read_csv('BTCUSDT3600.csv')
data['date'] = data['date'].astype('datetime64[s]')
data = data.set_index('date')
data = np.log(data)
data = data[data.index < '01-01-2020']
arr = data['close'].to_numpy()

n_pips = 5
lookback = 24
hold_period = 6

unique_pip_patterns = []
unique_pip_indices = []
cluster_centers = []
pip_clusters = []

cluster_signals = []
cluster_objs = []

long_signal = None
short_signal = None

selected_long = []
selected_short = []

fit_martin = None
perm_martins = []
        
data = None # Array of log closing prices to mine patterns
returns = None # Array of next log returns, concurrent with _data

In [9]:
def get_fit_martin():
    return fit_martin

def get_permutation_martins():
    return perm_martins

In [10]:
def plot_cluster_examples(self, candle_data: pd.DataFrame, cluster_i: int, grid_size: int = 5):
    plt.style.use('dark_background')
    fig, axs = plt.subplots(grid_size, grid_size)
    flat_axs = axs.flatten()
    for i in range(len(flat_axs)):
        if i >= len(self._pip_clusters[cluster_i]):
            break
        
        pat_i = self._unique_pip_indices[self._pip_clusters[cluster_i][i]]
        data_slice = candle_data.iloc[pat_i - self._lookback + 1: pat_i + 1]
        idx = data_slice.index
        plot_pip_x, plot_pip_y = find_pips(data_slice['close'].to_numpy(), self._n_pips, 3)
        
        pip_lines = []
        colors = []
        for line_i in range(self._n_pips - 1):
            l0 = [(idx[plot_pip_x[line_i]], plot_pip_y[line_i]), (idx[plot_pip_x[line_i + 1]], plot_pip_y[line_i + 1])]
            pip_lines.append(l0)
            colors.append('w')

        mpf.plot(data_slice, type='candle',alines=dict(alines=pip_lines, colors=colors), ax=flat_axs[i], style='charles', update_width_config=dict(candle_linewidth=1.75) )
        flat_axs[i].set_yticklabels([])
        flat_axs[i].set_xticklabels([])
        flat_axs[i].set_xticks([])
        flat_axs[i].set_yticks([])
        flat_axs[i].set_ylabel("")

    fig.suptitle(f"Cluster {cluster_i}", fontsize=32)
    plt.show()
